# Recent luminous extragalactic transients - joining tables

(Modified from an original by Joe Lyman). 
In this notebook we will be looking for luminous extragalactic transients by ustilising the information provided from the [Sherlock](https://qub-sherlock.readthedocs.io/en/latest/) contextual classifier. We will specifically look for transients that are luminous compared to their host galaxies, where there is a detection in the last 10 days. The aim is to quickly find superluminous supernovae candidates (very bright supernovae that often inhabit dwarf galaxies).

As part of the process we will be looking at joining tables in SQL queries, order to retrieve information about a given `object` from different tables.

First of all we need to do some imports, setup plotting, and connect to the ZTF database

In [1]:
import mysql.connector
import numpy as np
from astropy.table import Table
import astropy.coordinates as coord
import astropy.units as u
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from ztf import settings

In [2]:
msl = mysql.connector.connect(
    user=settings.DB_USER, 
    password=settings.DB_PASS, 
    host=settings.DB_HOST, database='ztf')
cursor = msl.cursor()

As an example, lets take a look at what information is provided by the Sherlock crossmatching:

In [3]:
names=['transient_object_id', 'catalogue_object_id', 'catalogue_table_id', 
                                   'separationArcsec', 'northSeparationArcsec', 'eastSeparationArcsec', 'id', 'z', 
                                   'scale', 'distance', 'distance_modulus', 'photoZ', 'photoZErr', 
                                   'association_type', 'dateCreated', 'physical_separation_kpc', 
                                   'catalogue_object_type', 'catalogue_object_subtype', 'association_rank', 
                                   'catalogue_table_name', 'catalogue_view_name', 'rank', 'rankScore', 
                                   'search_name', 'major_axis_arcsec', 'direct_distance', 'direct_distance_scale', 
                                   'direct_distance_modulus', 'raDeg', 'decDeg', 'original_search_radius_arcsec', 
                                   'catalogue_view_id', 'U', 'UErr', 'B', 'BErr', 'V', 'VErr', 'R', 'RErr', 
                                   'I', 'IErr', 'J', 'JErr', 'H', 'HErr', 'K', 'KErr', '_u', '_uErr', 
                                   '_g', '_gErr', '_r', '_rErr', '_i', '_iErr', '_z', '_zErr', '_y', '_yErr', 
                                   'G', 'GErr', 'unkMag', 'unkMagErr', 'dateLastModified', 'updated', 
                                   'classificationReliability', 'transientAbsMag', 'merged_rank']

cursor.execute("SELECT * FROM sherlock_crossmatches LIMIT 10")
results = cursor.fetchall()
table = Table(rows=results, names=names)
table

transient_object_id,catalogue_object_id,catalogue_table_id,separationArcsec,northSeparationArcsec,eastSeparationArcsec,id,z,scale,distance,distance_modulus,photoZ,photoZErr,association_type,dateCreated,physical_separation_kpc,catalogue_object_type,catalogue_object_subtype,association_rank,catalogue_table_name,catalogue_view_name,rank,rankScore,search_name,major_axis_arcsec,direct_distance,direct_distance_scale,direct_distance_modulus,raDeg,decDeg,original_search_radius_arcsec,catalogue_view_id,U,UErr,B,BErr,V,VErr,R,RErr,I,IErr,J,JErr,H,HErr,K,KErr,_u,_uErr,_g,_gErr,_r,_rErr,_i,_iErr,_z,_zErr,_y,_yErr,G,GErr,unkMag,unkMagErr,dateLastModified,updated,classificationReliability,transientAbsMag,merged_rank
int64,str30,int64,float64,float64,float64,int64,object,object,object,object,object,object,str7,object,object,str7,object,object,str18,str42,object,float64,str28,object,object,object,object,float64,float64,float64,int64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,int64,int64,object,object
1235661,232996065935604352/NCFO028307,71,0.07260430046265566,0.0726,-0.0008,6787825,None,None,None,None,None,None,VS,2018-11-02 07:38:47,None,star,multiple,None,GAIA/GSC,multiple,1,1002.0726043004627,multiple,None,None,None,None,64.3341565914,45.5942731734,0.0,36,None,None,15.3687,0.425363,14.7653,0.379915,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,15.9081842601,None,None,None,2018-11-02 07:38:47,1,1,None,None
1240537,1920670857505734144/23364861+4,71,0.21766104734733543,0.21677,-0.01975,6787826,None,None,None,None,None,None,VS,2018-11-02 07:38:47,None,star,multiple,None,GAIA/2MASS/GSC/NED,multiple,1,1002.2176610473473,multiple,None,None,None,None,354.20253545016783,40.811960814366444,0.0,36,None,None,15.5353,0.425884,14.8779,0.318287,None,None,None,None,13.611,0.026,13.219,0.032,13.157,0.029,None,None,None,None,None,None,None,None,None,None,None,None,14.7892467175,None,None,None,2018-11-02 07:38:47,0,1,None,None
1240537,1920670857505734144,71,0.21766104734733543,0.21677,-0.01975,6787827,None,None,None,None,None,None,VS,2018-11-02 07:38:47,None,star,None,None,Gaia DR1,tcs_view_star_gaia_dr1,None,1002.2176610473473,gaia star angular,None,None,None,None,354.20253545016783,40.811960814366444,100.0,36,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,14.7892467175,None,None,None,2018-11-02 07:38:47,0,1,None,1
1240537,23364861+4048430,1,0.23502377235760316,0.221,0.0798,6787828,None,None,None,None,None,None,VS,2018-11-02 07:38:47,None,star,None,None,2MASS PSC,tcs_view_star_2mass_psc_final,None,1002.2176610473473,2mass star angular,None,None,None,None,354.202572,40.811962,2.5,20,None,None,None,None,None,None,None,None,None,None,13.611,0.026,13.219,0.032,13.157,0.029,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2018-11-02 07:38:47,0,1,None,1
1240537,N07B003337,3,0.40430205594767765,0.24193,0.32395,6787829,None,None,None,None,None,None,VS,2018-11-02 07:38:47,None,star,0,None,GSC v2.3,tcs_view_star_guide_star_catalogue_v2_3,None,1002.2176610473473,GSC star 1 angular,None,None,None,None,354.202661592376,40.81196780144429,100.0,21,None,None,15.5353,0.425884,14.8779,0.318287,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,2018-11-02 07:38:47,0,1,None,1
1240537,2MASS J23364861+4048430,68,1.065696667539302,1.04,0.22,6787830,None,None,None,None,None,None,SN,2018-11-02 07:38:47,None,galaxy,UvS,None,NED,tcs_view_galaxy_like_ned_stream,None,1002.2176610473473,ned phot galaxy-like angular,None,None,None,None,354.202625,40.81219,10.0,39,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,Non

Of interest to us here is to select on objects that have a `sherlock_crossmatches.rank = 1` (i.e. the most likely) crossmatch object that has `sherlock_crossmatches.catalogue_object_type = 'galaxy'`. *(Note we could also try to select objects where the `sherlock_crossmatches.association_type = 'SN'` (supernova) if we wished, you can alter the query below as appropriate, if desired)* We will impose a cut on our objects that the must have at least 8 detections also, for us to assess the light curves - i.e. `objects.ncand > 8`.

We know what objects we want to select, now we need to decide the information (columns) we want to retrieve about the objects and their Sherlock crossmatches. Since we are comparing transient and host galaxy magnitudes, we want to retrieve the brightest `g` and `r` magnitudes from the `objects` table (confusingly named `magrmin` and `maggmin`, currently), and the `_r` and `_g` magnitudes from the `sherlock_crossmatches` table (note the underscores are used distinguish these columns from the other photometric systems in the table, e.g. `R`) - we should also grab the id, name and coordinates of our objects.

Since we require information from two different tables, and we want to retrieve that information for rows that are associated with the same object id, we must use a table `JOIN` and specifically and `ON` condition. You can see this used in the query below - there is plenty of SQL documentation online to find out more about these.

First we compute the current Julian Day from the Unix time:

In [4]:
import time, datetime
now = datetime.datetime.now()
print ("This notbook was run on " + now.isoformat())

jdnow = time.time()/86400 + 2440587.5
print ("and the Julian Date is {}".format(jdnow))

This notbook was run on 2019-11-27T15:29:18.846729
and the Julian Date is 2458815.1453570244


Now get the light curve information about all the objects we have found:

In [5]:
query = "SELECT objects.objectId, objects.maggmin, objects.magrmin, objects.ncandgp, "
query += "sherlock_crossmatches._g, sherlock_crossmatches._r "
query += "FROM objects,sherlock_crossmatches "
query += "WHERE objects.primaryId = sherlock_crossmatches.transient_object_id "
query += "AND objects.magrmin < 16 "
query += "AND sherlock_crossmatches.rank=1 "
query += "AND sherlock_crossmatches.catalogue_object_type = 'galaxy'"
query += "AND objects.jdmax > %f - 10.00000 " % jdnow
print(query)

SELECT objects.objectId, objects.maggmin, objects.magrmin, objects.ncandgp, sherlock_crossmatches._g, sherlock_crossmatches._r FROM objects,sherlock_crossmatches WHERE objects.primaryId = sherlock_crossmatches.transient_object_id AND objects.magrmin < 16 AND sherlock_crossmatches.rank=1 AND sherlock_crossmatches.catalogue_object_type = 'galaxy'AND objects.jdmax > 2458815.145357 - 10.00000 


In [6]:
cursor.execute(query)
results = cursor.fetchall()
table = Table(rows=results, names=["objectid", "magrmin", "maggmin", "ncandgp", "_g", "_r"])
table

objectid,magrmin,maggmin,ncandgp,_g,_r
str12,object,float64,int64,object,object
ZTF17aaaehsq,15.9903,15.2306,12,14.6526,13.8049
ZTF17aaaekxo,15.3571,14.6711,28,14.3708,13.792
ZTF17aaapzfr,17.5446,15.1379,10,16.1868,13.9937
ZTF17aaaewjv,14.5717,14.7221,0,14.877,13.224
ZTF17aaagrfw,16.0843,15.4192,21,14.6297,13.5482
ZTF17aaaiyep,15.879,15.5292,7,14.1313,13.6555
ZTF17aaajqkc,14.2403,14.0666,2,13.7364,12.9038
ZTF17aaaocpq,16.2702,14.6863,4,None,None
ZTF17aaasncs,15.9204,15.0895,29,14.7793,14.1447


In [ ]:
lum_objid = []
for filter_name, tran_mag, host_mag, c in zip(("g", "r"), ("maggmin", "magrmin"), ("_g", "_r"), ("C2", "C3")):
    # Remove those where there's no data
    m1 = table[tran_mag] != None
    m2 = table[host_mag] != None
    t = table[m1 & m2]
    # Calculate the magnitude difference
    magdiff = t[tran_mag] - t[host_mag]
    # Store those significantly brighter than their hosts
    lum_objid.extend(t["objectid"][magdiff <= -4])
    # Plot histogram
    plt.hist(magdiff, bins=25)
    plt.xlabel("transient - host mag")
    plt.ylabel("N")
    plt.title("{}".format(filter_name))
    plt.show()
    
# Remove duplicates (i.e. those added for both g and r filters)
lum_objid = tuple(set(lum_objid))

We can use our list of luminous object ids to grab their lightcurves by querying the `candidates` table.

In [8]:
print(lum_objid)

('ZTF19acthtyt', 'ZTF18acioquo', 'ZTF19acucowl', 'ZTF19acvwsns', 'ZTF19acufaqb', 'ZTF19acubzbw', 'ZTF19acvwxtp', 'ZTF19actjnxb', 'ZTF19actjnuk', 'ZTF19acvguwr', 'ZTF18aczunjk', 'ZTF19acvtskn', 'ZTF19acvgpdk', 'ZTF19actidpl', 'ZTF19acvwjhw', 'ZTF19acvjfks', 'ZTF19acucvmu', 'ZTF19acvxgqd', 'ZTF19actuluk', 'ZTF19actutzd', 'ZTF19acttsvs', 'ZTF19actuome', 'ZTF19aculaqw', 'ZTF19actfxmj', 'ZTF19acukgvt', 'ZTF19acughsd', 'ZTF19actjnna', 'ZTF19acvivod', 'ZTF19acukhvh', 'ZTF19acuevgq', 'ZTF19acudtpc', 'ZTF19acvwpxg', 'ZTF19acukwrc', 'ZTF19acvtukd', 'ZTF19acvwrtm', 'ZTF19acvybwr', 'ZTF19acvxoxh', 'ZTF19acvhimf', 'ZTF19acukelx', 'ZTF19actkppl', 'ZTF19actuuln', 'ZTF19acvxfzv', 'ZTF19actjmvc', 'ZTF19acucomr', 'ZTF19acvilgz', 'ZTF19acvwsmb', 'ZTF19actunxi', 'ZTF19actiaio', 'ZTF19acthzob', 'ZTF19actjnfy', 'ZTF19actjnmm', 'ZTF19acukulb', 'ZTF19acvhigl', 'ZTF19acvwsjl', 'ZTF19acvgvan', 'ZTF19actidxt', 'ZTF19acukuml', 'ZTF19acvjrtb', 'ZTF19actumqz', 'ZTF19acviqmc', 'ZTF17aaakibm', 'ZTF19actjnqy', 'ZTF19a

In [9]:
# Get the light curves
query = "SELECT objectid, jd-{} AS ago, magpsf, sigmapsf, fid "
query += "FROM candidates WHERE objectid IN {};".format(jdnow, lum_objid)
print(query)
cursor.execute(query)
result = cursor.fetchall()

SELECT objectid, jd-{} AS ago, magpsf, sigmapsf, fid FROM candidates WHERE objectid IN 2458815.1453570244;


ProgrammingError: 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '} AS ago, magpsf, sigmapsf, fid FROM candidates WHERE objectid IN 2458815.145357' at line 1

In [ ]:
ztf_lc_res = Table(rows=result, names=("objectid", "ago", "magpsf", "sigmapsf", "fid")).group_by("objectid")
ztf_lc_res

For each luminous object, plot the light curve

In [ ]:
for objid in lum_objid:
    lc = ztf_lc_res[ztf_lc_res["objectid"] == objid]
    if len(lc) < 8:
        continue
    lcg = lc[[lc["fid"] == 1]]
    plt.errorbar(lcg["ago"], lcg["magpsf"], yerr=lcg["sigmapsf"], c="C2")
    lcr = lc[[lc["fid"] == 2]]
    plt.errorbar(lcr["ago"], lcr["magpsf"], yerr=lcr["sigmapsf"], c="C3")
    plt.xlabel("days in the past")
    plt.ylabel("psf mag")
    plt.title("{}".format(objid))
    plt.gca().invert_yaxis()
    plt.show()